# Volatility and Risk

We've seen that the volatility is measured by the average squared deviation from the mean, which is the standard deviation.

Let's read the sample returns that we've been working with.

In [2]:
import pandas as pd
prices = pd.read_csv("../Homework/sample_prices.csv")
returns = prices.pct_change()
returns

,BLUE,ORANGE
0,NaN,NaN
1,0.023621,0.039662
2,-0.021807,-0.033638
3,-0.031763,0.082232
4,0.034477,0.044544
5,0.037786,-0.026381
6,-0.011452,-0.049187
7,0.032676,0.117008
8,-0.012581,0.067353
9,0.029581,0.078249


Notice that the first set of returns are NaN, which is Pandas way of saying that it's an NA. We can drop that row using the `.dropna()` method.

In [3]:
returns = returns.dropna()
returns

,BLUE,ORANGE
1,0.023621,0.039662
2,-0.021807,-0.033638
3,-0.031763,0.082232
4,0.034477,0.044544
5,0.037786,-0.026381
6,-0.011452,-0.049187
7,0.032676,0.117008
8,-0.012581,0.067353
9,0.029581,0.078249
10,0.006151,-0.168261


Let's compute the standard deviation from first principals:

In [4]:
deviations = returns - returns.mean()
squared_deviations = deviations**2
mean_squared_deviations = squared_deviations.mean()

import numpy as np

volatility = np.sqrt(mean_squared_deviations)
volatility

BLUE      0.022957
ORANGE    0.076212
dtype: float64

Let's see if we get the same answer when we use the built-in `.std()` method.

In [5]:
returns.std()

BLUE      0.023977
ORANGE    0.079601
dtype: float64

Why don't they match? Because, by default, the `.std()` method computes the _sample standard deviation_ which means that it uses the denominator of $n-1$. On the other hand, we computed the _population_ standard deviation, which uses a numerator of $n$. Since the observed returns are thought of as observed samples from a distribution, it is probably more accurate to use the denominator of $n-1$, so let's redo our calculation to see if we get the same number.

To get the number of observations, we can use the `.shape` attribute of a DataFrame that returns a tuple of the number of rows and columns.

In [6]:
returns.shape

(12, 2)

Just as we can with a list, we can access the elements of a tuple using an index, starting at 0. Therefore, to get the number of rows in the DataFrame, we extract the 0th element of the tuple.

In [7]:
number_of_obs = returns.shape[0]
mean_squared_deviations = squared_deviations.sum()/(number_of_obs-1)
volatility = np.sqrt(mean_squared_deviations)
volatility

BLUE      0.023977
ORANGE    0.079601
dtype: float64

In [8]:
returns.std()

BLUE      0.023977
ORANGE    0.079601
dtype: float64

# Annualizing Volatility

We annualize volatility by scaling (multiplying) it by the square root of the number of periods per observation

Therefore, to annualize the volatility of a monthly series, we muiltiply it by the square root of 12. Instead of using the `np.sqrt()` we can raise it to the power of $0.5$

In [9]:
annualized_vol = returns.std()*(12**0.5)
annualized_vol

BLUE      0.083060
ORANGE    0.275747
dtype: float64

# Risk Adjusted Returns

Let's get beyond the sample data series and start working with some real data. Read in the monthly returns of a set of 10 portfolios formed on market caps, or market equities of the companies. Of the 10 portfolios, we only want to look at the largest cap and the smallest cap companies:

In [33]:
me_m = pd.read_csv("../Homework/Portfolios.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)
me_m.describe()

,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,Dec 2,Dec 3,Dec 4,Dec 5,Dec 6,Dec 7,Dec 8,Dec 9,Hi 10
count,0.0,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000,1110.000000
mean,NaN,1.495811,1.150847,0.982261,1.590396,1.240811,1.157072,1.070108,0.943432,1.759757,1.339730,1.269450,1.210811,1.146838,1.166018,1.082360,1.056856,0.998387,0.886982
std,NaN,9.121026,7.066472,5.815717,9.719729,7.928957,7.048299,6.393745,5.632515,10.628818,9.117535,8.192778,7.723223,7.212501,6.932748,6.592381,6.262746,5.953184,5.390026
min,NaN,-30.990000,-30.990000,-30.510000,-30.990000,-31.470000,-32.130000,-29.310000,-30.400000,-29.140000,-33.020000,-31.170000,-31.770000,-31.440000,-32.830000,-29.110000,-30.730000,-32.310000,-28.430000
25%,NaN,-2.557500,-2.045000,-1.985000,-2.695000,-2.387500,-2.170000,-2.010000,-1.917500,-2.735000,-2.730000,-2.497500,-2.355000,-2.277500,-2.137500,-2.012500,-2.070000,-1.985000,-1.702500
50%,NaN,1.265000,1.455000,1.310000,1.190000,1.475000,1.510000,1.455000,1.230000,1.045000,1.185000,1.400000,1.430000,1.485000,1.440000,1.325000,1.485000,1.365000,1.140000
75%,NaN,4.717500,4.650000,3.960000,4.830000,4.717500,4.767500,4.337500,3.837500,4.995000,5.025000,4.737500,4.775000,4.677500,4.577500,4.375000,4.277500,3.967500,3.765000
max,NaN,100.370000,58.400000,44.240000,112.600000,81.640000,56.530000,50.640000,41.630000,134.290000,106.160000,83.080000,80.210000,58.000000,57.180000,51.900000,49.460000,47.530000,37.790000


In [35]:
#rename the columns
cols = ['Lo 20', 'Hi 20']
ret = me_m[cols]
ret

,Lo 20,Hi 20
192607,-0.57,3.33
192608,3.84,2.33
192609,-0.48,-0.09
192610,-3.29,-2.95
192611,-0.55,3.16
...,...,...
201808,3.09,2.49
201809,-2.04,0.19
201810,-10.52,-7.41
201811,-2.78,2.49


In [36]:
returns= ret.loc['2009':'2016']
returns.head()

,Lo 20,Hi 20
200901,-1.35,-6.50
200902,-11.44,-10.95
200903,13.10,10.46
200904,20.97,17.62
200905,11.88,8.06


Note that the data is already given in percentages (i.e 4.5 instead of 0.045) and we typically want to use the actual numbers (i.e. 0.045 instead of 4.5) so we should divide the raw data from the file by 100.

In [37]:
returns = returns/100

In [38]:
returns.plot()

In [39]:
returns.columns = ['SmallCap', 'LargeCap']

In [40]:
returns.plot()

In [41]:
annualized_vol = returns.std()*np.sqrt(12)
annualized_vol

SmallCap    0.201164
LargeCap    0.163028
dtype: float64

We can now compute the annualized returns as follows:

In [42]:
n_months = returns.shape[0]
return_per_month = (returns+1).prod()**(1/n_months) - 1
return_per_month

SmallCap    0.013378
LargeCap    0.013214
dtype: float64

In [43]:
annualized_return = (return_per_month + 1)**12-1

In [44]:
annualized_return = (returns+1).prod()**(12/n_months) - 1
annualized_return

SmallCap    0.172888
LargeCap    0.170619
dtype: float64

In [45]:
annualized_return/annualized_vol

SmallCap    0.859438
LargeCap    1.046566
dtype: float64

In [46]:
riskfree_rate = 0.03
excess_return = annualized_return - riskfree_rate
sharpe_ratio = excess_return/annualized_vol
sharpe_ratio

SmallCap    0.710306
LargeCap    0.862548
dtype: float64